# Level 4.5: Agentic RAG with reference Eval

This tutorial presents an example of evaluating an agentic RAG in LLama-Stack using the reference implementation. 
Please refer to `# Level4_agentic_RAG.ipynb` [notebook](../rag_agentic/notebooks/Level4_RAG_agent.ipynb)
for details on how to initialize the agent and the knowledge search RAG tool provided by Llama Stack.

## Overview

This tutorial covers the following steps:
1. Connecting to a llama-stack server.
2. Indexing a collection of documents in a vector DB for later retrieval.
3. Initializing the agent capable of retrieving content from vector DB via tool use.
4. Evaluating the agent responses against a reference set of Q&A.
5. Reporting the evaluation results and its statistical relevance.

## Case study
For the purpose of this training, we are going to use the fictional company 
[Parasol Financial](https://www.redhat.com/en/blog/ai-insurance-industry-insights-red-hat-summit-2024), and the provided
[training documents](https://github.com/jharmison-redhat/parasol-financial-data/).

A sample Q&A document is available as a [reference](./data/parasol-financial-data_qac.yaml). 
This predefined question and answer pairs have beeen generated using [docling-sdg](https://github.com/docling-project/docling-sdg),
an IBM set of tools to create artificial data from documents, leveraging generative AI and Docling's parsing capabilities.

## Prerequisites

Before starting, ensure you have a running instance of the Llama Stack server (local or remote) with at least one preconfigured vector DB. For more information, please refer to the corresponding [Llama Stack tutorials](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

The `openai` inference provider is required if you intend to use an OpenAI model for judging purposes, like `openai/gpt-4o`. In this case, the 
`OPENAI_API_KEY` env variable must be configured into the Llama Stack server.

## Setting the Environment Variables

Use the [`.env.example`](../../../.env.example) to create a new file called `.env` and ensure you add all the relevant environment variables below.

In addition to the environment variables listed in the ["Getting Started" notebook](../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb), the following should be provided for this demo to run:
 - `LLM_AS_JUDGE_MODEL_ID`: the model to use as the judge to evaluate the agent responses. Must be one of the models defined in Llama Stack.
 - `VDB_PROVIDER`: the vector DB provider to be used. Must be supported by Llama Stack. For this demo, we use Milvus Lite which is our preferred solution.
 - `VDB_EMBEDDING`: the embedding model to be used for ingestion and retrieval. For this demo, we use all-MiniLM-L6-v2.
 - `VDB_EMBEDDING_DIMENSION` (optional): the dimension of the embedding. Defaults to 384.
 - `VECTOR_DB_CHUNK_SIZE` (optional): the chunk size for the vector DB. Defaults to 512.

## 1. Setting Up the Environment
We will start with a few imports needed for this demo only.

In [1]:
import numpy as np
import pandas as pd
import time
import uuid

from rich.pretty import pprint

from IPython.display import display_markdown

from llama_stack_client import Agent, AgentEventLogger, RAGDocument
from llama_stack_client.lib.agents.event_logger import EventLogger

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [2]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv(override=True)

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient
# to override the judge model
from llama_stack.providers.inline.scoring.llm_as_judge.scoring_fn.fn_defs.llm_as_judge_405b_simpleqa import (
    llm_as_judge_405b_simpleqa,
)

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

remote = os.getenv("REMOTE", "True")

if remote == "False":
    local_port = os.getenv("LOCAL_SERVER_PORT", 8321)
    base_url = f"http://localhost:{local_port}"
else: # any value non equal to 'False' will be considered as 'True'
    base_url = os.getenv("REMOTE_BASE_URL")

client = LlamaStackClient(
    base_url=base_url,
    provider_data=None
)
    
print(f"Connected to Llama Stack server @ {base_url}")

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = os.getenv("INFERENCE_MODEL_ID")

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "True")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

# The Q&A file
QNA_FILE = './data/parasol-financial-data_qac.yaml'
# The number of rows to consider
MAX_QNA_ROWS = 50
# Set to True to enable display of evaluation results
EVAL_DEBUG = False
llm_as_judge_model = os.getenv("LLM_AS_JUDGE_MODEL_ID")
llm_as_judge_405b_simpleqa_params = llm_as_judge_405b_simpleqa.params.model_copy()
# Override the default model
# To update the scoring params, we need to provide all the settings, including the defaults
llm_as_judge_405b_simpleqa_params.judge_model = llm_as_judge_model

# Convert the model dump to a dictionary
scoring_params = llm_as_judge_405b_simpleqa_params.model_dump()
scoring_params['aggregation_functions']=['categorical_count']

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")
print(f"Eval Parameters:\n\tJudge Model: {llm_as_judge_model}\n\tQ&A file: {QNA_FILE}\n\tMax rows: {MAX_QNA_ROWS}")

Connected to Llama Stack server @ http://localhost:8321
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: True
Eval Parameters:
	Judge Model: openai/gpt-4o
	Q&A file: ./data/parasol-financial-data_qac.yaml
	Max rows: 50


Finally, we will initialize the document collection to be used for RAG ingestion and retrieval.

In [3]:
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
display_markdown(f"Registered vector DB **{vector_db_id}**", raw=True)


Registered vector DB **test_vector_db_4ab507b9-8618-4d92-bebf-73c1566578c2**

## 2. Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle fetching, conversion and chunking of the documents' content.
- Perform a sample query to verify the response is retrieved from the relevant documents.

In [4]:
# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=os.getenv("VDB_EMBEDDING"),
    embedding_dimension=int(os.getenv("VDB_EMBEDDING_DIMENSION", 384)),
    provider_id=os.getenv("VDB_PROVIDER"),
)

# ingest the documents into the newly created document collection
urls = [
    "flexible_enhanced_checking/flexible_enhanced_checking.md",
    "flexible_savings/flexible_savings.md",
    "flexible_premier_checking/flexible_premier_checking.md",
    "flexible_core_checking/flexible_core_checking.md",
    "policies/online_service_agreement.md",
    "enablement/customer_interactions_resource_guide.md",
    "enablement/banking_essentials_resource_guide.md",
    "flexible_money_market_savings/flexible_money_market_savings.md",
    "flexible_checking/flexible_checking.md",
]
documents = [
    RAGDocument(
        document_id=f"{url.split('/')[-1]}",
        content=f"https://raw.githubusercontent.com/jharmison-redhat/parasol-financial-data/main/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512)),
)

In [5]:
# Query documents
results = client.tool_runtime.rag_tool.query(
    vector_db_ids=[vector_db_id],
    content="What is the Parasol Financial Withdrawal Limit Fee and Transaction Limitations for Flexible Money Market Savings",
)
results.metadata['document_ids']

['flexible_money_market_savings.md',
 'flexible_money_market_savings.md',
 'flexible_savings.md',
 'flexible_savings.md',
 'online_service_agreement.md']

## 3. Defining reusable functions
Define reusable Python functions to use during the execution of the evaluation jobs.


In [6]:
def accuracy_from_categorical_count(response):
    """
    Computes the evaluation accuracy from the responses of the `llm-as-judge::405b-simpleqa`
    scoring function.

    Expected responses are:
    ```
    A: CORRECT
    B: INCORRECT
    C: NOT_ATTEMPTED
    ```
    The accuracy is computed as: <number of responses of type `A`> / <number of responses> * 100
    """
    # Evaluate numerical score
    correct_answers = sum(
        [
            count
            for cat, count in response.scores["llm-as-judge::405b-simpleqa"]
            .aggregated_results["categorical_count"]["categorical_count"]
            .items()
            if cat == "A"
        ]
    )
    num_of_scores = len(response.scores["llm-as-judge::405b-simpleqa"].score_rows)
    return correct_answers / num_of_scores * 100

In [7]:
def _run_eval(use_rag: bool):
    """
    Runs the evaluation function for the benchmark indicated by the global variable `qna_benchmark_id`.
    A new agent is created for every function call: in case `use_rag` is set to `True`, the `knowledge_search` tool is defined
    to implement the RAG workflow.
    The global variables `model_id` and `vector_db_id` are also requested.

    Params:
        use_rag: whether to run a RAG workflow or not.
    Returns:
        the `Job` associated to the evaluation function.
    """

    from httpx import Timeout

    if use_rag == True:
        instructions = "You are a helpful assistant. You must use the knowledge search tool to answer user questions."
        tools = [
            dict(
                name="builtin::rag",
                args={
                    "vector_db_ids": [
                        vector_db_id
                    ],  # list of IDs of document collections to consider during retrieval
                },
            )
        ]
    else:
        instructions = "You are a helpful assistant."
        tools = []

    agent_config = {
        "model": model_id,
        "instructions": instructions,
        "sampling_params": sampling_params,
        "toolgroups": tools,
    }

    _job = client.eval.run_eval(
        benchmark_id=qna_benchmark_id,
        benchmark_config={
            "num_examples": MAX_QNA_ROWS,
            "scoring_params": {
                "llm-as-judge::405b-simpleqa": scoring_params,
            },
            "eval_candidate": {
                "type": "agent",
                "config": agent_config,
            },
        },
        timeout=Timeout(MAX_QNA_ROWS * 30),  # Allow for 30s per Q&A
    )
    return _job

In [8]:
def _get_eval_reponse(_job):
    """
    Returns the `EvalResponse` instance for the given `_job`.

    Params:
        `job_id`: The evaluation `Job`.
    Returns:
        The `EvalResponse` for the given `_job`
    """
    status = client.eval.jobs.status(
        benchmark_id=qna_benchmark_id, job_id=_job.job_id
    ).status
    while status != "completed":
        print(f"Job status is {status}")
        sleep(1)
        status = client.eval.jobs.status(
            benchmark_id=qna_benchmark_id, job_id=_job.job_id
        ).status
    print(f"Job status is {status}")
    _eval_response = client.eval.jobs.retrieve(
        benchmark_id=qna_benchmark_id, job_id=_job.job_id
    )

    return _eval_response

In [9]:
def to_label(score_row):
    """
    Returns the display label for the given `score_row`.
    """
    grades = {'A': 'CORRECT', 'B': 'INCORRECT', 'C': 'NOT_ATTEMPTED'}
    score = score_row.get('score', str(score_row))
    return grades.get(score,  f'UNKNOWN {score}')

In [10]:
def numeric_scores(response):
    """
    Converts the computed scores in a numeric array, where scores `A` are evaluated to 1
    and all the others to `0`.
    """
    def category_to_number(category):
        if category == 'A':
            return 1
        return 0

    return [category_to_number(score_row['score']) for score_row in response.scores['llm-as-judge::405b-simpleqa'].score_rows]

In [11]:
def permutation_test_for_paired_samples(scores_a, scores_b, iterations=10_000):
    """
    Performs a permutation test of a given statistic on provided data.
    """

    from scipy.stats import permutation_test


    def _statistic(x, y, axis):
        return np.mean(x, axis=axis) - np.mean(y, axis=axis)

    result = permutation_test(
        data=(scores_a, scores_b),
        statistic=_statistic,
        n_resamples=iterations,
        alternative='two-sided',
        permutation_type='samples'
    )
    return float(result.pvalue)

In [12]:
def print_stats_significance(scores_a, scores_b, label_a, label_b):
    mean_score_a = np.mean(scores_a)
    mean_score_b = np.mean(scores_b)

    p_value = permutation_test_for_paired_samples(scores_a, scores_b)
    print(model_id)
    print(f" {label_a:<50}: {mean_score_a:>10.4f}")
    print(f" {label_b:<50}: {mean_score_b:>10.4f}")
    print(f" {'p_value':<50}: {p_value:>10.4f}")
    print()

    if p_value < 0.05:
        print("p_value<0.05 so this result is statistically significant")
        # Note that the logic below if wrong if the mean scores are equal, but that can't be true if p<1.
        higher_model_id = (
            label_a
            if mean_score_a >= mean_score_b
            else label_b
        )
        print(f"You can conclude that {higher_model_id} generation is better on data of this sort")
    else:
        import math

        print("p_value>=0.05 so this result is NOT statistically significant.")
        print(
            f"You can conclude that there is not enough data to tell which is better."
        )
        num_samples = len(scores_a)
        margin_of_error = 1 / math.sqrt(num_samples)
        print(
            f"Note that this data includes {num_samples} questions which typically produces a margin of error of around +/-{margin_of_error:.1%}."
        )
        print(f"So the two are probably roughly within that margin of error or so.")

## 4. Creating an evaluation Dataset
- Load the Q&A file as a Pandas DataFrame.
- Transform the dataset to a schema suitable for LLS evaluations.
- Register a new Dataset.
- Register a Benchmark using the Dataset and the `llm-as-judge::405b-simpleqa` scoring function.

In [13]:
with open(QNA_FILE, "r") as f:
    qnas_df = pd.read_json(f, lines=True)
pd.set_option("display.max_colwidth", None)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [14]:
from llama_stack.apis.inference import UserMessage
import json
import random

qna_dataset_rows = []

chat_completion_input = UserMessage(content="")
for i in range(len(qnas_df)):
    qna = {}
    qna["input_query"] = qnas_df.iloc[i]["question"]
    qna["expected_answer"] = qnas_df.iloc[i]["answer"]

    chat_completion_input.content = qna["input_query"]
    qna["chat_completion_input"] = json.dumps([chat_completion_input.model_dump()])

    qna_dataset_rows.append(qna)

random.shuffle(qna_dataset_rows)
qna_dataset_df = pd.DataFrame(qna_dataset_rows)
qna_dataset_df.head()

,input_query,expected_answer,chat_completion_input
0,What is the phone number provided for contacting Parasol Financial?,800.867.5309,"[{""role"": ""user"", ""content"": ""What is the phone number provided for contacting Parasol Financial?"", ""context"": null}]"
1,Why might the company include exceptions to their liability in the agreement?,"The company includes exceptions to their liability to protect themselves from situations that are beyond their control or not their fault, such as insufficient funds in the customer's account, known service malfunctions, uncontrollable events like natural disasters, and delays caused by third parties.","[{""role"": ""user"", ""content"": ""Why might the company include exceptions to their liability in the agreement?"", ""context"": null}]"
2,What are the main purposes for which the Bank shares anonymized transaction information?,"The Bank shares anonymized transaction information to facilitate participation in the rewards program, present offers of interest, and administer benefits and rewards with participating merchants, third parties, and card networks.","[{""role"": ""user"", ""content"": ""What are the main purposes for which the Bank shares anonymized transaction information?"", ""context"": null}]"
3,What happens if the person to whom you are sending money does not enroll with Zelle within 14 days?,The transfer will be canceled.,"[{""role"": ""user"", ""content"": ""What happens if the person to whom you are sending money does not enroll with Zelle within 14 days?"", ""context"": null}]"
4,What might be a consequence of opting out of security alerts?,"A consequence of opting out of security alerts might be that the user will not receive any notifications about potential security issues with their credit card, business line of credit, or debit card, which could leave them unaware of unauthorized transactions or other security concerns.","[{""role"": ""user"", ""content"": ""What might be a consequence of opting out of security alerts?"", ""context"": null}]"


In [15]:
qna_dataset_id = f"test_dataset_{uuid.uuid1()}"
_ = client.datasets.register(
    purpose="eval/messages-answer",
    source={
        "type": "rows",
        "rows": qna_dataset_rows,
    },
    dataset_id=qna_dataset_id,
)
display_markdown(f"Registered dataset **{qna_dataset_id}**", raw=True)


Registered dataset **test_dataset_0eea3690-29a7-11f0-a45b-4a70c355aff9**

In [16]:
qna_benchmark_id = f"test_benchmark_{uuid.uuid1()}"
client.benchmarks.register(
    benchmark_id=qna_benchmark_id,
    dataset_id=qna_dataset_id,
    scoring_functions=["llm-as-judge::405b-simpleqa"],
)
display_markdown(f"Registered benchmark **{qna_benchmark_id}**", raw=True)


Registered benchmark **test_benchmark_0f4b9f52-29a7-11f0-a45b-4a70c355aff9**

## 5. LLM Eval without RAG
- Create an agent configuration without the `knowledge_search` tool.
- Run the evaluation function with the current configuration.

In [17]:
start = time.time()

without_rag_responses = {}
_job = _run_eval(use_rag=False)
# pprint(_job)
_eval_response = _get_eval_reponse(_job)
if EVAL_DEBUG == True:
    pprint(_eval_response)

print(
    f"Evaluation of {MAX_QNA_ROWS} Q&A workflows completed in {time.time() - start:.3f} seconds"
)
display_markdown(
    f"**Computed accuracy is {accuracy_from_categorical_count(_eval_response)}%**",
    raw=True,
)
without_rag_responses[model_id] = _eval_response

Job status is completed
Evaluation of 50 Q&A workflows completed in 387.319 seconds


**Computed accuracy is 40.0%**

## 6. LLM Eval with RAG
- Create an agent configuration with the `knowledge_search` tool.
- Run the evaluation function with the current configuration.

In [18]:
start = time.time()

rag_responses = {}
_job = _run_eval(use_rag=True)
# pprint(_job)
_eval_response = _get_eval_reponse(_job)
if EVAL_DEBUG == True:
    pprint(_eval_response)

print(
    f"Evaluation of {MAX_QNA_ROWS} Q&A workflows completed in {time.time() - start:.3f} seconds"
)
display_markdown(
    f"**Computed accuracy is {accuracy_from_categorical_count(_eval_response)}%**",
    raw=True,
)
rag_responses[model_id] = _eval_response

retrieved_contexts = sum(
    [1 for r in rag_responses[model_id].generations if "context" in r]
)
display_markdown(
    f"**RAG knowledge search tool used in {retrieved_contexts} of ({MAX_QNA_ROWS}) agentic calls**",
    raw=True,
)

Job status is completed
Evaluation of 50 Q&A workflows completed in 264.498 seconds


**Computed accuracy is 64.0%**

**RAG knowledge search tool used in 48 of (50) agentic calls**

## 4. Reporting
- Aggregated accuracy.
- Individual scores and responses.
- Statistical Significance.

In [19]:
pd_responses = {}
pd_responses['questions'] = [qna_dataset_rows[i]['input_query'] for i in range(MAX_QNA_ROWS)]
pd_responses['expected'] = [qna_dataset_rows[i]['expected_answer'] for i in range(MAX_QNA_ROWS)]

pd_accuracies = {}
df_accuracies = pd.DataFrame.from_dict({
    'Model': without_rag_responses.keys(),
    'Accuracy without RAG': [accuracy_from_categorical_count(without_rag_responses[model_id]) for model_id in without_rag_responses.keys()],
    'Accuracy with RAG': [accuracy_from_categorical_count(rag_responses[model_id]) for model_id in rag_responses.keys()]})
df_accuracies.style.hide()

Model,Accuracy without RAG,Accuracy with RAG
granite32-8b,40.000000,64.000000


In [20]:
report_data = {}
ratings_data = {}
responses_data = {}

report_data['Question'] = [qna_dataset_rows[i]['input_query'] for i in range(MAX_QNA_ROWS)]
ratings_data['Question'] = report_data['Question']
responses_data['Question'] = report_data['Question']
report_data['Expected Answer'] = [qna_dataset_rows[i]['expected_answer'] for i in range(MAX_QNA_ROWS)]
responses_data['Expected Answer'] = report_data['Expected Answer']
for model_id in without_rag_responses.keys():
    report_data[f'Rating without RAG'] = [to_label(score_row) for score_row in without_rag_responses[model_id].scores['llm-as-judge::405b-simpleqa'].score_rows]
    report_data[f'Answer without RAG'] = [g['generated_answer'] for g in without_rag_responses[model_id].generations]
    report_data[f'Rating with RAG'] = [to_label(score_row) for score_row in rag_responses[model_id].scores['llm-as-judge::405b-simpleqa'].score_rows]
    report_data[f'Answer with RAG'] = [g['generated_answer'] for g in rag_responses[model_id].generations]
    
    ratings_data[f'Rating without RAG'] = report_data[f'Rating without RAG']
    responses_data[f'Answer without RAG'] = report_data[f'Answer without RAG']
    ratings_data[f'with RAG RAG Rating'] = report_data[f'Rating with RAG']
    responses_data[f'Answer with RAG'] = report_data[f'Answer with RAG']

In [21]:
from IPython.display import HTML


display_markdown(f"**Response Comparison for model {model_id}**", raw=True)
report_df = pd.DataFrame.from_dict(report_data)
HTML(report_df.head().to_html(index=False))


**Response Comparison for model granite32-8b**

Question,Expected Answer,Rating without RAG,Answer without RAG,Rating with RAG,Answer with RAG
What is the phone number provided for contacting Parasol Financial?,800.867.5309,NOT_ATTEMPTED,"I'm sorry for any inconvenience, but as an AI, I don't have real-time access to specific databases or the ability to browse the internet. Therefore, I can't provide the current contact number for Parasol Financial. I recommend visiting their official website or doing a quick online search to find the most accurate and up-to-date contact information.",CORRECT,The phone number provided for contacting Parasol Financial is 800.867.5309.
Why might the company include exceptions to their liability in the agreement?,"The company includes exceptions to their liability to protect themselves from situations that are beyond their control or not their fault, such as insufficient funds in the customer's account, known service malfunctions, uncontrollable events like natural disasters, and delays caused by third parties.",CORRECT,"Companies often include exceptions to their liability in agreements for several reasons:\n\n1. **Risk Management**: Every business operation involves some level of risk. By specifying exceptions, companies can clearly outline the circumstances under which they won't be held responsible. This helps in managing and mitigating potential risks.\n\n2. **Legal Protection**: These exceptions serve as a legal safeguard. They help protect the company from unforeseen or unusual circumstances that might lead to claims or lawsuits. \n\n3. **Clarity and Transparency**: Exceptions clearly define the boundaries of the company's responsibilities. This transparency can help prevent misunderstandings or disputes down the line.\n\n4. **Standard Industry Practice**: In many industries, it's common to include such clauses in contracts. They're often part of standard form agreements, and their inclusion can be seen as a standard business practice.\n\n5. **Cost Control**: By limiting their liability, companies can control their potential financial exposure. This can be particularly important for smaller businesses or startups.\n\n6. **Force Majeure**: This is a specific type of exception that absolves a company from liability due to unforeseeable circumstances beyond their control, such as natural disasters, wars, or strikes.\n\n7. **Intellectual Property**: In agreements involving intellectual property, exceptions might be included to protect the company's own IP or to limit liability related to third-party IP.\n\nRemember, the specific reasons can vary based on the nature of the business, the type of agreement, and the jurisdiction in which it's being conducted. It's always important to have legal advice when interpreting or negotiating such clauses.",CORRECT,"The company might include exceptions to their liability in the agreement for several reasons:\n\n1. **Risk Management**: By specifying exceptions, the company can clearly outline the circumstances under which they won't be held responsible. This helps in managing potential risks and liabilities.\n\n2. **Legal Protection**: Exceptions can provide legal protection for the company. They can limit the company's liability in situations that are beyond their control or due to circumstances that were disclosed upfront.\n\n3. **Clarity and Transparency**: Exceptions clearly define the boundaries of the company's responsibilities. This transparency can help in setting clear expectations for both parties involved in the agreement.\n\n4. **Fairness**: In some cases, certain risks or damages might be inherent to the service or product being provided. Excluding liability for these would be fair, as the company can't reasonably be expected to cover these.\n\n5. **Cost Control**: By limiting liability, companies can control their potential financial exposure. This can help in managing costs and maintaining profitability.\n\nThese are general reasons and the specific rationale would depend on the nature of 

In [22]:
display_markdown("**Statistical Significance (without Vs with RAG generations)**", raw= True)
print_stats_significance(numeric_scores(without_rag_responses[model_id]), numeric_scores(rag_responses[model_id]), "accuracy without RAG", "accuracy with RAG")

**Statistical Significance (without Vs with RAG generations)**

granite32-8b
 accuracy without RAG                              :     0.4000
 accuracy with RAG                                 :     0.6400
 p_value                                           :     0.0040

p_value<0.05 so this result is statistically significant
You can conclude that accuracy with RAG generation is better on data of this sort


## Key Takeaways
This tutorial demonstrates how to evaluate an agentic workflow, without and without RAG tool, using the Llama Stack reference implementation.
We do so by initializing an agent, with optional access to the RAG tool, then invoking the agent evaluation against a predefined reference of sample Q&A. 
Please check out our [complementary tutorial](../rag_agentic/notebooks/Level4_RAG_agent.ipynb) for an agentic RAG example.